In [1]:
import numpy as np
import pandas as pd
import time, datetime
import sys, os, os.path
import scipy, scipy.signal

from datetime import date, datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, accuracy_score, \
                            confusion_matrix, balanced_accuracy_score, \
                            classification_report
import matplotlib.pyplot as plt

# from patsy import cr
# from pprint import pprint
# from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
import NASA_plot_core as ncp

2024-06-14 14:31:52.704063: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
NASA_dir_base = "/Users/hn/Documents/01_research_data/NASA/"
meta_dir = NASA_dir_base + "0000_parameters/"
train_TS_dir_base = NASA_dir_base + "VI_TS/"

data_part_of_shapefile_dir = NASA_dir_base + "data_part_of_shapefile/"


In [4]:
meta_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
meta = pd.read_csv(meta_dir+"evaluation_set.csv")
meta_moreThan10Acr=meta[meta.ExctAcr>10]
print (meta.shape)
print (meta_moreThan10Acr.shape)
meta.head(2)

(6340, 8)
(3539, 8)


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [5]:
training_set_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data_Oct17/"
ground_truth_labels = pd.read_csv(training_set_dir+"groundTruth_labels_Oct17_2022.csv")
print ("Unique Votes: ", ground_truth_labels.Vote.unique())
print (len(ground_truth_labels.ID.unique()))
ground_truth_labels.head(2)

Unique Votes:  [2 1]
3160


,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


In [9]:
csv_data_parts = os.listdir(data_part_of_shapefile_dir)
csv_data_parts = [x for x in csv_data_parts if x.endswith("csv")]
csv_data_parts

['FranklinYakima2018.csv',
 'irriigated_SF_data_concatenated.csv',
 'all_eastern_centroid.csv',
 'Eastern_2018_centroid.csv',
 'Eastern_2015_centroid.csv',
 'Walla2015.csv',
 'AdamBenton2016.csv',
 'all_SF_data_concatenated.csv',
 'Eastern_2016_centroid.csv',
 'Eastern_2017_centroid.csv',
 'Monterey2014.csv',
 'Grant2017.csv']

In [10]:
irriigated_SF_data_concatenated = pd.read_csv(data_part_of_shapefile_dir + "irriigated_SF_data_concatenated.csv")
irriigated_SF_data_concatenated.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams


In [11]:
irriigated_SF_data_concatenated.county.unique()

array(['Adams', 'Benton', 'Franklin', 'Yakima', 'Grant', 'Walla Walla'],
      dtype=object)

In [23]:
irriigated_SF_data_concatenated["survey_year"] = pd.to_datetime(irriigated_SF_data_concatenated["LstSrvD"], 
                                                                format='mixed').dt.year

irriigated_SF_data_concatenated.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016


In [24]:
irriigated_SF_data_concatenated.head(3)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016
2,51383_WSDA_SF_2016,wheat,113,113.432725,center pivot,2016/09/13 00:00:00,wsda,Adams,2016


In [29]:
irriigated_SF_data_concatenated["SF_year"] = irriigated_SF_data_concatenated["ID"].str.split("_", expand=True)[3]
irriigated_SF_data_concatenated["SF_year"] = irriigated_SF_data_concatenated["SF_year"].astype(int)
irriigated_SF_data_concatenated.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year,SF_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016,2016


In [31]:
(irriigated_SF_data_concatenated["SF_year"] == irriigated_SF_data_concatenated["survey_year"]).sum()

44850

In [32]:
irriigated_SF_data_concatenated.shape

(53582, 10)

In [39]:
# limit to large fields
irriigated_SF_data_concatenated = irriigated_SF_data_concatenated[irriigated_SF_data_concatenated.ExctAcr > 10]

In [40]:
irriigated_SF_data_concatenated.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year,SF_year
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016,2016
2,51383_WSDA_SF_2016,wheat,113,113.432725,center pivot,2016/09/13 00:00:00,wsda,Adams,2016,2016


In [42]:
len(list(irriigated_SF_data_concatenated.CropTyp.unique()))

103